In [1]:
from neural_astar_jax.utils import data
from neural_astar_jax.utils import training
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
from importlib import reload
from neural_astar_jax.planner import astar, differentiable_astar
reload(data)
reload(training)
reload(differentiable_astar)
reload(astar)
import optax
from flax.training.train_state import TrainState

In [2]:
planner = astar.NeuralAstar(is_training=True, search_step_ratio=0.25)
train_loader = data.MazeDataLoader(filename="../../planning-datasets/data/mpd/mazes_032_moore_c8.npz", split="train", batch_size=100)
val_loader = data.MazeDataLoader(filename="../../planning-datasets/data/mpd/mazes_032_moore_c8.npz", split="val", batch_size=1)
trainer = training.Trainer(planner=planner, train_loader=train_loader, val_loader=val_loader, val_every_n_steps=1)

In [3]:
state = trainer.fit(jax.random.PRNGKey(0))

step=00, train_loss=0.0527, val_loss=0.0760, p_opt=0.3400, p_exp=0.4449, h_mean=0.3854
best model updated (h_mean=0.3854 > best_h_mean=-inf)
step=01, train_loss=0.0635, val_loss=0.0753, p_opt=0.3500, p_exp=0.4454, h_mean=0.3920
best model updated (h_mean=0.3920 > best_h_mean=0.3854)
step=02, train_loss=0.0537, val_loss=0.0757, p_opt=0.3200, p_exp=0.4446, h_mean=0.3722
step=03, train_loss=0.0570, val_loss=0.0753, p_opt=0.3300, p_exp=0.4487, h_mean=0.3803
step=04, train_loss=0.0590, val_loss=0.0732, p_opt=0.3500, p_exp=0.4628, h_mean=0.3986
best model updated (h_mean=0.3986 > best_h_mean=0.3920)
step=05, train_loss=0.0558, val_loss=0.0730, p_opt=0.3200, p_exp=0.4647, h_mean=0.3790
step=06, train_loss=0.0616, val_loss=0.0730, p_opt=0.3400, p_exp=0.4655, h_mean=0.3930
step=07, train_loss=0.0538, val_loss=0.0727, p_opt=0.3400, p_exp=0.4651, h_mean=0.3928
step=08, train_loss=0.0605, val_loss=0.0725, p_opt=0.3500, p_exp=0.4688, h_mean=0.4008
best model updated (h_mean=0.4008 > best_h_mean=0.3